# import all the necessary libraries

In [1]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# save the open ai key in environmental variable

In [2]:
OPENAI_API_KEY = "your_api_key"
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

C:\Users\padma\AppData\Local\Temp\ipykernel_18840\1703113971.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


# Database setup

In [3]:
host = 'localhost'
port = '3306'
username = 'root'
#password = ''
database_schema = 'fresher'
mysql_uri = f"mysql+pymysql://{username}@{host}:{port}/{database_schema}"
#mysql_uri = f"mysql+pymysql://{username:password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri, include_tables=['p6_uk_bank_customers'],sample_rows_in_table_info=2)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [4]:
def retrieve_from_db(query: str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

In [7]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    
    system_message = """You are a database expert specializing in customer segmentation, balance prediction, or understanding customer demographics.
        your task is to answer users' questions by providing relevant information from a database of training data.
        Tha database contaims the following information for each individual:
        -Customer ID: A unique identifier for each customer(discrete)
        -Name: The first name of the customer(categorical)
        -Surname: The last name of the customer(categorical)
        -Gender: The gender of the customer (discrete) (Male/Female)
        -Age: The age of the customer(discrete)
        -Region: The region where the customer resides(categorical)
        -Job Classification: The type of job ( White Collar, Blue Collar)(categorical)
        -Date Joined: The date the customer joined the bank(categorical)
        -Balance: The current balance in the customer's account(continous)

        Example:

        Input:
        Which region has the most customers?

        context:
        Analyzing customer distribution across regions helps the bank identify its strongest customer base geographically.
        
        Output:
        The region with the most customers is England, with 2,159 customers
        """
    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        
        Context:
        {db_context}
        
        Output:
        """
    )
    messages = [
      SystemMessage(content=system_message),
      human_qry_template.format(human_input=query, db_context=db_context)
    ]
    response = llm(messages).content
    return response

# ask queries

In [8]:
generate("Provide names of all people from England")



> Entering new SQLDatabaseChain chain...
Provide names of all people from England
SQLQuery:SELECT `Name`, `Surname` 
FROM p6_uk_bank_customers 
WHERE `Region` = 'England'
LIMIT 5;
SQLResult: [('Simon', 'Walsh'), ('Liam', 'Brown'), ('Deirdre', 'Pullman'), ('Dorothy', 'Thomson'), ('Dominic', 'Parr')]
Answer:Simon Walsh, Liam Brown, Deirdre Pullman, Dorothy Thomson, Dominic Parr
> Finished chain.


C:\Users\padma\AppData\Local\Temp\ipykernel_18840\2179357571.py:42: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages).content


'The names of all people from England are Simon Walsh, Liam Brown, Deirdre Pullman, Dorothy Thomson, and Dominic Parr.'

In [9]:
generate("Provide count of all Male people")



> Entering new SQLDatabaseChain chain...
Provide count of all Male people
SQLQuery:SELECT COUNT(`Customer ID`) AS Male_Count
FROM p6_uk_bank_customers
WHERE Gender = 'Male';
SQLResult: [(2165,)]
Answer:There are 2165 Male people in the database.
> Finished chain.


'There are 2,165 Male customers in the database.'

In [10]:
generate("Provide count of all female who are from Wales")



> Entering new SQLDatabaseChain chain...
Provide count of all female who are from Wales
SQLQuery:SELECT COUNT(`Customer ID`) AS `FemaleCount`
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND Region = 'Wales';
SQLResult: [(282,)]
Answer:There are 282 female customers from Wales.
> Finished chain.


'There are 282 female customers from Wales.'

In [19]:
generate("Provide names of all people who are white color and age above 35")



> Entering new SQLDatabaseChain chain...
Provide names of all people who are white color and age above 35
SQLQuery:SELECT `Name`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar' AND `Age` > 35
LIMIT 5;
SQLResult: [('Liam',), ('Dominic',), ('Dominic',), ('Benjamin',), ('Nicholas',)]
Answer:Liam, Dominic, Benjamin, Nicholas
> Finished chain.


'The names of people who are white collar and above the age of 35 are Dominic and Nicholas.'

In [11]:
generate("Provide count of people with White color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people with White color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `White Collar Count`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar';
SQLResult: [(1955,)]
Answer:There are 1955 people with White Collar jobs.
> Finished chain.


'There are 1,955 people with White Collar jobs.'

In [12]:
generate("Provide count of people with Blue Color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people with Blue Color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `Blue Collar Count`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'Blue Collar';
SQLResult: [(1049,)]
Answer:Blue Collar Count: 1
> Finished chain.


'The count of people with Blue Collar jobs is 1.'

In [13]:
generate("Provide count of people whose balance is above 5000")



> Entering new SQLDatabaseChain chain...
Provide count of people whose balance is above 5000
SQLQuery:SELECT COUNT(`Customer ID`) AS `Count of People`
FROM p6_uk_bank_customers
WHERE `Balance` > 5000;
SQLResult: [(3690,)]
Answer:There are 3690 people whose balance is above 5000.
> Finished chain.


'The count of people whose balance is above 5000 is 3690.'

In [14]:
generate("Provide minimum balance and name, job classification")



> Entering new SQLDatabaseChain chain...
Provide minimum balance and name, job classification
SQLQuery:SELECT `Name`, `Job Classification`, MIN(`Balance`) AS `Minimum Balance`
FROM p6_uk_bank_customers
GROUP BY `Name`, `Job Classification`
ORDER BY `Minimum Balance`
LIMIT 5;
SQLResult: [('Jane', 'White Collar', Decimal('11.52')), ('Olivia', 'White Collar', Decimal('21.03')), ('Warren', 'Blue Collar', Decimal('69.01')), ('Megan', 'Blue Collar', Decimal('69.78')), ('Stewart', 'Blue Collar', Decimal('77.46'))]
Answer:The customers with the minimum balance and their corresponding job classifications are:
1. Jane with a minimum balance of 11.52 in the White Collar job classification
2. Olivia with a minimum balance of 21.03 in the White Collar job classification
3. Warren with a minimum balance of 69.01 in the Blue Collar job classification
4. Megan with a minimum balance of 69.78 in the Blue Collar job classification
5. Stewart with a minimum balance of 77.46 in the Blue Collar job class

'The customers with the minimum balance and their corresponding job classifications are:\n1. Jane with a minimum balance of 11.52 in the White Collar job classification\n2. Olivia with a minimum balance of 21.03 in the White Collar job classification'

In [15]:
generate("provide maximum balance and name of the person")



> Entering new SQLDatabaseChain chain...
provide maximum balance and name of the person
SQLQuery:SELECT `Name`, `Balance` 
FROM p6_uk_bank_customers 
ORDER BY `Balance` DESC 
LIMIT 1;
SQLResult: [('Dorothy', Decimal('183467.70'))]
Answer:The person with the maximum balance is Dorothy with a balance of 183467.70.
> Finished chain.


'The person with the maximum balance is Dorothy with a balance of 183467.70.'

In [16]:
generate("Provide average age of people with white color job")



> Entering new SQLDatabaseChain chain...
Provide average age of people with white color job
SQLQuery:SELECT AVG(`Age`) AS `Average Age`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar';
SQLResult: [(Decimal('35.2793'),)]
Answer:The average age of people with a white collar job is 35.2793 years.
> Finished chain.


'The average age of people with a white collar job is 35.2793 years.'

In [17]:
generate("Provide count of people from Wales and doing white color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people from Wales and doing white color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `Count`
FROM p6_uk_bank_customers
WHERE Region = 'Wales' AND `Job Classification` = 'White Collar';
SQLResult: [(305,)]
Answer:There are 305 people from Wales doing white collar jobs.
> Finished chain.


'The count of people from Wales doing white collar jobs is 305.'

In [18]:
generate("Provide count of all female people who are with White color job")



> Entering new SQLDatabaseChain chain...
Provide count of all female people who are with White color job
SQLQuery:SELECT COUNT(`Customer ID`) AS Female_White_Collar_Count
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND `Job Classification` = 'White Collar';
SQLResult: [(1010,)]
Answer:There are 1010 female customers with a White Collar job.
> Finished chain.


'There are 1010 female customers with a White Collar job.'